# State Farm Distracted Driver Detection - implemented from scratch using VGG16 weights

### First get standard boilerplate imports out of the way

In [1]:
%matplotlib inline

In [2]:
from __future__ import division,print_function

#import os, json
from time import time
#from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

### Define path and batch_size for desired data source

In [3]:
batch_size = 150

In [4]:
#path = "data/"
#test_path = "data/test/"

path = "data/sample/"
test_path = "data/sample/test/"


## Create VGG16 model and load pre-trained weights from ImageNet

In [5]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### Preprocess routine to center each image pixel around dataset RGB mean value, and reverse axis from RGB -> BGR

In [6]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

In [7]:
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]

In [8]:
model = Sequential()


### Add Input Layer for images with dimension 244 x 244

In [9]:
model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))

### Add Convolutional Layers

In [10]:
def ConvBlock(mymodel, layers, filters):
    for i in range(layers):
        mymodel.add(ZeroPadding2D((1,1)))
        mymodel.add(Convolution2D(filters, 3, 3, activation='relu'))
    mymodel.add(MaxPooling2D((2,2), strides=(2,2)))

In [11]:
ConvBlock(model, 2, 64)
ConvBlock(model, 2, 128)
ConvBlock(model, 3, 256)
ConvBlock(model, 3, 512)
ConvBlock(model, 3, 512)

### Add Fully Connected Layers

In [12]:
def FCBlock(mymodel):
    mymodel.add(Dense(4096, activation='relu'))
    mymodel.add(Dropout(0.5))

In [13]:
model.add(Flatten())
FCBlock(model)
FCBlock(model)

### Add final output Dense Layer

In [14]:
model.add(Dense(1000, activation='softmax'))

### Download and load pre-trained model weights from Imagenet

In [15]:
fname = 'vgg16.h5'
baseurl = 'http://www.platform.ai/models/'
local_weights = get_file(fname, baseurl+fname, cache_subdir='models')

In [16]:
model.load_weights(local_weights)

## Finetune VGG16 by replacing last layer with a Dense layer

In [17]:
model.pop()

Make all remaining layers read-only, since we want to leverage all the pre-trained weights

In [18]:
for layer in model.layers: layer.trainable=False

Get batch of data for training and validation

In [19]:
def get_batches(path, gen=image.ImageDataGenerator(rotation_range=10), shuffle=True, batch_size=8, class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(224,224), color_mode='rgb', class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [20]:
train_batches = get_batches(path+'train', batch_size=batch_size)
valid_batches = get_batches(path+'valid', batch_size=batch_size)

Found 750 images belonging to 10 classes.
Found 250 images belonging to 10 classes.


### Now add new Dense layer with number of classes we are training with, and compile.

In [21]:
model.add(Dense(train_batches.nb_class, activation='softmax'))

In [22]:
model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

### Now fit the model with training and validation batches

In [ ]:
model.optimizer.lr=1e-3
model.fit_generator(train_batches, 
                    samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=6,
                    validation_data=valid_batches,
                    nb_val_samples=valid_batches.nb_sample
                   )

Epoch 1/6
750/750 [==============================] - 27s - loss: 2.2977 - acc: 0.3400 - val_loss: 1.9581 - val_acc: 0.4200
Epoch 2/6
750/750 [==============================] - 25s - loss: 2.1361 - acc: 0.3893 - val_loss: 1.9078 - val_acc: 0.4160
Epoch 3/6
300/750 [===========>..................] - ETA: 10s - loss: 1.9370 - acc: 0.4233

In [ ]:
model.optimizer.lr=1e-5
model.fit_generator(train_batches, 
                    samples_per_epoch=train_batches.nb_sample,
                    nb_epoch=10,
                    validation_data=valid_batches,
                    nb_val_samples=valid_batches.nb_sample
                   )

In [ ]:
model.save_weights('data/sample/models/try2.h5')

### Generate test predictions

In [ ]:
test_batches = get_batches(test_path, shuffle=False, batch_size=2*batch_size, class_mode=None)

In [ ]:
test_predictions = model.predict_generator(test_batches, test_batches.nb_sample)

### Generate Kaggle submission file

So to play it safe, we use a sneaky trick to round down our edge predictions.
Swap all ones with .95 and all zeros with .05

In [ ]:
test_predictions = test_predictions.clip(min=0.05, max=0.95)

Prepare filenames by stripping path

In [ ]:
test_filenames = [filename[filename.find('/')+1:] for filename in test_batches.filenames]

Prepare the submission data matrix the dumb way...

In [ ]:
subm1 = np.zeros(test_batches.nb_sample, dtype=[('filename', 'S128'), ('c0', float), ('c1', float), ('c2', float), ('c3', float), ('c4', float), ('c5', float), ('c6', float), ('c7', float), ('c8', float), ('c9', float)])

subm1['filename'] = test_filenames

subm1['c0'] = test_predictions[:,0]
subm1['c1'] = test_predictions[:,1]
subm1['c2'] = test_predictions[:,2]
subm1['c3'] = test_predictions[:,3]
subm1['c4'] = test_predictions[:,4]
subm1['c5'] = test_predictions[:,5]
subm1['c6'] = test_predictions[:,6]
subm1['c7'] = test_predictions[:,7]
subm1['c8'] = test_predictions[:,8]
subm1['c9'] = test_predictions[:,9]

Write Kaggle submission file

In [ ]:
submission_file_name = 'submission_' + str(int(time())) + '.csv'
np.savetxt(submission_file_name, subm1, fmt='%s,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f', header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9', comments='')

### Visualize the prediction by plotting a few

Define function to convert Category ID encoded via bit vector to Category Description from Kaggle's data description page:
* c0: safe driving 
* c1: texting - right 
* c2: talking on the phone - right 
* c3: texting - left 
* c4: talking on the phone - left 
* c5: operating the radio 
* c6: drinking 
* c7: reaching behind 
* c8: hair and makeup 
* c9: talking to passenger

In [ ]:
def convertCategetoryIDToTextDescription(category_id):
    return {
        0: 'safe driving',
        1: 'texting - right',
        2: 'talking on the phone - right',
        3: 'texting - left',
        4: 'talking on the phone - left',
        5: 'operating the radio',
        6: 'drinking',
        7: 'reaching behind',
        8: 'hair and makeup',
        9: 'talking to passenger'
    }.get(category_id, 'unknown')

Use argmax to find index with highest probability.
Index happens to be same value as State Farm Cateogry Id itself

In [ ]:
test_pred_winning_category_id = test_predictions.argmax(axis=1)
test_pred_winning_categories = [convertCategetoryIDToTextDescription(x) for x in test_pred_winning_category_id]
test_pred_winning_prob = np.choose(test_pred_winning_category_id, test_predictions.transpose())

Helper function to plot images for chosen indices

In [ ]:
from utils import plots
    
def plots_idx(idx, path, filenames, categories, probabilities):
    my_image_ids = np.array([int(filenames[i][filenames[i].find('_')+1:filenames[i].find('.')]) for i in idx])
    my_categories = [categories[i] for i in idx]
    my_probabilities = [probabilities[i] for i in idx]
    
    # label = <iamge id>=<probability>:<category>
    my_labels = ["{}={:0.2f}:{}".format(i, j, k) for i,j,k in zip(my_image_ids, my_probabilities, my_categories)]
    
    plots([image.load_img(path + filenames[i]) for i in idx], titles=my_labels)    

In [ ]:
n_view = 4

idx = np.random.permutation(range(0,test_batches.nb_sample))[:n_view]
plots_idx(idx, test_path, test_batches.filenames, test_pred_winning_categories, test_pred_winning_prob)
